In [ ]:
# Get us into the correct directory to import our fb_auth class
%cd ../lwll_api

In [ ]:
# By doing this import, it instantiates a class which reads our appropriate firebase account creds so that
# we can import firebase_admin directly after and be authenticated to the database of interest
from lwll_api.classes.fb_auth import fb_store

In [3]:
from firebase_admin import auth

```
uid – User ID to assign to the newly created user (optional).
display_name – The user’s display name (optional).
email – The user’s primary email (optional).
email_verified – A boolean indicating whether or not the user’s primary email is verified (optional).
phone_number – The user’s primary phone number (optional).
photo_url – The user’s photo URL (optional).
password – The user’s raw, unhashed password. (optional).
disabled – A boolean indicating whether or not the user account is disabled (optional).
app – An App instance (optional).
```

In [5]:
# For simplicity we instantiate the password as a unique id, UI functionality to change this will be built in later
# at the creation of UI to watch sessions

lwll_user_list = [
    {'team': 'ExampleTeam', 'email': 'example@team.com', 'password': '12345', 'uid': 'someuid', 'permissions':['user']},
    {'team': 'ExampleGovTeam', 'email': 'exampleGov@team.com', 'password': '12345', 'uid': 'someotheruid', 'permissions': ['user', 'gov']},

]

In [22]:
# Example getting user by uid
# u = auth.get_user('401e4afa-7892-4ed6-8afe-ef636c1993a4')


In [24]:
#u.display_name

Step 1 is to run this loop to create the users if they don't exist

In [ ]:
for user in lwll_user_list:
    try:
        auth.create_user(uid=user['uid'], display_name=user['team'], password=user['password'])
        fb_store.collection('Users').document(user['uid']).set({'team':user['team'], 'permissions':user['permissions']})
        print(f"User `{user['team']} : {user['uid']}` created!")
    except auth.UidAlreadyExistsError:
        print(f"User `{user['team']} : {user['uid']}` already exsists, skipping...")

Step 2

These accounts were created anonymously the first time, so let's add an email to them with our admin sdk so that people will be able to authenticate against them to sign in and view their stats.

In [ ]:
for user in lwll_user_list:
    print(f"Updating user: {user['uid']}")
    auth.update_user(user['uid'], email= user['email'], email_verified=True, disabled=False, password=user['password'])
    fb_store.collection('Users').document(user['uid']).update({'active':True})
    